<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/database.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Importing Data from Kafka into SingleStore using Pipelines</h1>
    </div>
</div>

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Note</b></p>
        <p>This notebook can be run on a Free Starter Workspace. To create a Free Starter Workspace navigate to <tt>Start</tt> using the left nav. You can also use your existing Standard or Premium workspace with this Notebook.</p>
    </div>
</div>

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Input Credentials</b></p>
        <p>Define the <b>BOOTSTRAP_SERVER</b>, <b>PORT</b>, <b>TOPIC</b>,<b>SASL_USERNAME</b>,<b>SASL_MECHANISM</b>,<b>SECURITY_PROTOCOL</b>, and <b>SASL_PASSWORD</b> variables below for integration, replacing the placeholder values with your own.</p>
    </div>
</div>

In [1]:
BOOTSTRAP_SERVER = 'bootstrap-server-url'
PORT = kafka-broker-port
TOPIC = 'kafka-topic'
SASL_USERNAME = 'username'
SASL_MECHANISM = 'sasl-mechanism'
SECURITY_PROTOCOL = 'security-proptocol'
SASL_PASSWORD = 'password'

This notebook demonstrates how to create a sample table in SingleStore, set up a pipeline to import data from an Kafka topic, and run queries on the imported data. It is designed for users who want to integrate Kafka data with SingleStore and explore the capabilities of pipelines for efficient data ingestion.

<h3>Pipeline Flow Illustration</h3>

<img src=https://singlestoreloaddata.s3.ap-south-1.amazonaws.com/images/LoadDataKafka.png width="100%" hight="50%"/>

##  Creating Table in SingleStore

Start by creating a table that will hold the data imported from Kafka.

In [2]:
%%sql
/* Feel free to change table name and schema */

CREATE TABLE IF NOT EXISTS my_table (
    id INT,
    name VARCHAR(255),
    age INT,
    address TEXT,
    created_at TIMESTAMP
);

## Create a Pipeline to Import Data from Kafka

You'll need to create a pipeline that pulls data from Kafka topic into this table. This example assumes you have a JSON Message in your Kakfa topic.

<i>Ensure that:
You have access to the Kafka topic.
Proper IAM roles or access keys are configured in SingleStore.
The JSON message has a structure that matches the table schema.</i>

Using these identifiers and keys, execute the following statement.

In [3]:
%%sql
CREATE OR REPLACE PIPELINE kafka_import_pipeline AS LOAD DATA KAFKA '{{BOOTSTRAP_SERVER}}:{{PORT}}/{{TOPIC}}'
CONFIG '{
  "sasl.username": "{{SASL_USERNAME}}",
  "sasl.mechanism": "{{SASL_MECHANISM}}",
  "security.protocol": "{{SECURITY_PROTOCOL}}",
  "ssl.ca.location": "/etc/pki/ca-trust/extracted/pem/tls-ca-bundle.pem"
}'
CREDENTIALS '{
  "sasl.password": "{{SASL_PASSWORD}}"
}'
INTO TABLE my_table
FORMAT JSON ;

## Start the Pipeline

To start the pipeline and begin importing the data from the S3 bucket:

In [4]:
%%sql
START PIPELINE kafka_import_pipeline;

## Select Data from the Table

Once the data has been imported, you can run a query to select it:

In [5]:
%%sql
SELECT * FROM my_table LIMIT 10;

### Check if all data of the data is loaded

In [6]:
%%sql
SELECT count(*) FROM my_table

## Conclusion

We have shown how to insert data from a Amazon S3 using `Pipelines` to SingleStoreDB. These techniques should enable you to
integrate your Amazon S3 with SingleStoreDB.

## Clean up

Remove the '#' to uncomment and execute the queries below to clean up the pipeline and table created.

#### Drop Pipeline

In [7]:
%%sql
#STOP PIPELINE kafka_import_pipeline;

#DROP PIPELINE kafka_import_pipeline;

#### Drop Data

In [8]:
%%sql
#DROP TABLE my_table;

<div id="singlestore-footer" style="background-color: rgba(194, 193, 199, 0.25); height:2px; margin-bottom:10px"></div>
<div><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-grey.png" style="padding: 0px; margin: 0px; height: 24px"/></div>